In [ ]:
import numpy as np
import anndata as ad

# 假设adata是你的原始AnnData对象
# 假设要合成的点位索引在python列表indices中

# 计算坐标的平均值
coords_mean = np.mean(adata.obsm['spatial'][indices, :], axis=0)

# 合并基因表达数据，这里我们取平均值
expr_mean = np.mean(adata.X[indices, :], axis=0)

# 创建一个新的点位数据矩阵和坐标矩阵
new_data_matrix = np.vstack([adata.X, expr_mean])
new_spatial_coords = np.vstack([adata.obsm['spatial'], coords_mean])

# 创建一个新的AnnData对象或更新原有对象
# 这里创建一个新的示例
new_adata = ad.AnnData(X=new_data_matrix)
new_adata.obsm['spatial'] = new_spatial_coords

# 如果有其他注释需要复制，也可以在这里操作
# 例如，复制基因名
new_adata.var_names = adata.var_names

# 注意，合成点位后可能需要更新.obs等其他注释


In [5]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import anndata as ad


def reduce_cell(slice:ad.AnnData):
    # 假设slice是原始的AnnData对象

    # 计算空间点位的距离矩阵
    coords = slice.obsm['spatial']
    nbrs = NearestNeighbors(n_neighbors=10, algorithm='auto').fit(coords)
    distances, indices = nbrs.kneighbors(coords)

    # 初始化新的数据结构
    new_coords = []
    new_data_matrix = []

    # 对于每组最近邻，计算坐标和基因表达的均值
    for idx_group in indices:
        # 计算坐标的均值
        group_coords = coords[idx_group].mean(axis=0)
        new_coords.append(group_coords)
        
        # 计算基因表达的均值
        group_expr = slice.X[idx_group].mean(axis=0)
        new_data_matrix.append(group_expr)

    # 将列表转换为numpy数组
    new_coords = np.array(new_coords)
    new_data_matrix = np.vstack(new_data_matrix)

    # 创建新的AnnData对象
    new_slice = ad.AnnData(X=new_data_matrix)
    new_slice.obsm['spatial'] = new_coords
    new_slice.var_names = slice.var_names

    print(new_slice)


data_matrix = np.random.randint(0,3,size=(9,3))
#print(data_matrix)
slice = ad.AnnData(X=data_matrix)
    


[[1 0 0]
 [0 2 2]
 [0 2 0]
 [2 1 0]
 [1 0 2]
 [1 2 1]
 [0 1 1]
 [1 0 1]
 [2 1 0]]
